# Develop conversion methods between dependent species and endmembers

In [1]:
import numpy as np
from os import path
import pandas as pd
import scipy.optimize as opt
from scipy import optimize
import scipy.linalg as lin
import scipy as sp
import sys
import sympy as sym

from collections import OrderedDict as odict

import matplotlib.pyplot as plt

import sunkenhull as hull
from thermoengine.model import GeoCompDB, SysComp

from scipy.spatial import ConvexHull
import scipy.interpolate as interp

import ternary

In [2]:
import time
from IPython import display
time.sleep(.001)

Required ENKI modules (ignore the error message from Rubicon running under Python 3.6+)

In [3]:
from thermoengine import coder, core, phases, model, equilibrate

In [4]:
modelDB = model.Database(database='Berman')

## Test miscibility gap for Feldspar

In [5]:
Fsp = modelDB.get_phase('Fsp')
Cpx = modelDB.get_phase('Cpx')
Oam = modelDB.get_phase('Oam')
Cam = modelDB.get_phase('Cam')

In [6]:
Cpx.props['species_name']

array(['diopside', 'clinoenstatite', 'hedenbergite', 'alumino-buffonite',
       'buffonite', 'essenite', 'jadeite', 'fe-aluminobuffonite',
       'fe-buffonite', 'ca-tschermaks', 'mg-tschermaks', 'fe-tschermaks',
       'acmite', 'ferrosilite'], dtype='<U19')

In [7]:
Cpx.endmember_names

array(['diopside', 'clinoenstatite', 'hedenbergite', 'alumino-buffonite',
       'buffonite', 'essenite', 'jadeite'], dtype='<U17')

## Manual Development of species <-> endmember conversions

In [8]:
phs = Cpx

In [9]:
elem_comp_spec = phs.props['species_elms']
elem_mask = np.any(elem_comp_spec>0,axis=0)
elem_stoic_spec = elem_comp_spec[:,elem_mask]


In [10]:
elems = core.chem.PERIODIC_ORDER[elem_mask]
species_names = phs.props['species_name']
species_elem_stoic = pd.DataFrame(elem_stoic_spec,columns=elems, index=species_names)
species_elem_stoic

,O,Na,Mg,Al,Si,Ca,Ti,Fe
diopside,6.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0
clinoenstatite,6.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0
hedenbergite,6.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0
alumino-buffonite,6.0,0.0,0.5,1.0,1.0,1.0,0.5,0.0
buffonite,6.0,0.0,0.5,0.0,1.0,1.0,0.5,1.0
essenite,6.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
jadeite,6.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0
fe-aluminobuffonite,6.0,0.0,0.0,1.0,1.0,1.0,0.5,0.5
fe-buffonite,6.0,0.0,0.0,0.0,1.0,1.0,0.5,1.5
ca-tschermaks,6.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0


In [11]:
endmem_elem_stoic = pd.DataFrame(phs.props['element_comp'][:,elem_mask],
                            columns=elems, index=phs.endmember_names)
endmem_elem_stoic

,O,Na,Mg,Al,Si,Ca,Ti,Fe
diopside,6.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0
clinoenstatite,6.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0
hedenbergite,6.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0
alumino-buffonite,6.0,0.0,0.5,1.0,1.0,1.0,0.5,0.0
buffonite,6.0,0.0,0.5,0.0,1.0,1.0,0.5,1.0
essenite,6.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
jadeite,6.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0


In [12]:
TOL=1e-12
species_stoic = []

for iname, ielem_stoic in species_elem_stoic.iterrows():
    iresults = np.linalg.lstsq(endmem_elem_stoic.T, ielem_stoic, rcond=None)
    istoic = iresults[0]
    istoic[np.abs(istoic)<TOL] = 0
    species_stoic.append(istoic)

In [13]:
species_stoic = pd.DataFrame(species_stoic, columns=endmem_elem_stoic.index, 
                             index=species_elem_stoic.index)

In [14]:
species_stoic

,diopside,clinoenstatite,hedenbergite,alumino-buffonite,buffonite,essenite,jadeite
diopside,1.0,0.0,0.0,0.0,0.0,0.0,0.0
clinoenstatite,0.0,1.0,0.0,0.0,0.0,0.0,0.0
hedenbergite,0.0,0.0,1.0,0.0,0.0,0.0,0.0
alumino-buffonite,0.0,0.0,0.0,1.0,0.0,0.0,0.0
buffonite,0.0,0.0,0.0,0.0,1.0,0.0,0.0
essenite,0.0,0.0,0.0,0.0,0.0,1.0,0.0
jadeite,0.0,0.0,0.0,0.0,0.0,0.0,1.0
fe-aluminobuffonite,-0.5,0.0,0.5,1.0,0.0,0.0,0.0
fe-buffonite,-0.5,0.0,0.5,0.0,1.0,0.0,0.0
ca-tschermaks,0.0,0.0,0.0,1.0,-1.0,1.0,0.0


In [15]:
species_stoic.T

,diopside,clinoenstatite,hedenbergite,alumino-buffonite,buffonite,essenite,jadeite,fe-aluminobuffonite,fe-buffonite,ca-tschermaks,mg-tschermaks,fe-tschermaks,acmite,ferrosilite
diopside,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.5,-0.5,0.0,-1.0,-2.0,0.0,-2.0
clinoenstatite,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
hedenbergite,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,1.0,0.0,2.0
alumino-buffonite,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,-1.0,0.0
buffonite,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,1.0,0.0
essenite,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
jadeite,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Built-in stoichiometry matrices

In [16]:
phs.species_stoic

,diopside,clinoenstatite,hedenbergite,alumino-buffonite,buffonite,essenite,jadeite
diopside,1.0,0.0,0.0,0.0,0.0,0.0,0.0
clinoenstatite,0.0,1.0,0.0,0.0,0.0,0.0,0.0
hedenbergite,0.0,0.0,1.0,0.0,0.0,0.0,0.0
alumino-buffonite,0.0,0.0,0.0,1.0,0.0,0.0,0.0
buffonite,0.0,0.0,0.0,0.0,1.0,0.0,0.0
essenite,0.0,0.0,0.0,0.0,0.0,1.0,0.0
jadeite,0.0,0.0,0.0,0.0,0.0,0.0,1.0
fe-aluminobuffonite,-0.5,0.0,0.5,1.0,0.0,0.0,0.0
fe-buffonite,-0.5,0.0,0.5,0.0,1.0,0.0,0.0
ca-tschermaks,0.0,0.0,0.0,1.0,-1.0,1.0,0.0


In [17]:
phs.endmember_elem_comp

,O,Na,Mg,Al,Si,Ca,Ti,Fe
diopside,6.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0
clinoenstatite,6.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0
hedenbergite,6.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0
alumino-buffonite,6.0,0.0,0.5,1.0,1.0,1.0,0.5,0.0
buffonite,6.0,0.0,0.5,0.0,1.0,1.0,0.5,1.0
essenite,6.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
jadeite,6.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0


In [18]:
phs.species_elem_comp

,O,Na,Mg,Al,Si,Ca,Ti,Fe
diopside,6.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0
clinoenstatite,6.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0
hedenbergite,6.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0
alumino-buffonite,6.0,0.0,0.5,1.0,1.0,1.0,0.5,0.0
buffonite,6.0,0.0,0.5,0.0,1.0,1.0,0.5,1.0
essenite,6.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
jadeite,6.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0
fe-aluminobuffonite,6.0,0.0,0.0,1.0,1.0,1.0,0.5,0.5
fe-buffonite,6.0,0.0,0.0,0.0,1.0,1.0,0.5,1.5
ca-tschermaks,6.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0


# Conversions

In [19]:
mol_spec = pd.Series(np.random.rand(species_stoic.shape[0]), index=species_stoic.index)
mol_spec /= mol_spec.sum()
mol_spec

diopside               0.028584
clinoenstatite         0.039667
hedenbergite           0.083546
alumino-buffonite      0.091271
buffonite              0.102915
essenite               0.154600
jadeite                0.012275
fe-aluminobuffonite    0.010965
fe-buffonite           0.081971
ca-tschermaks          0.066724
mg-tschermaks          0.094206
fe-tschermaks          0.096339
acmite                 0.077018
ferrosilite            0.059920
dtype: float64

In [20]:
# Species -> Endmems
mol_endmem = species_stoic.T.dot(mol_spec)
mol_endmem

diopside            -0.424608
clinoenstatite       0.290132
hedenbergite         0.346192
alumino-buffonite    0.282486
buffonite            0.004635
essenite             0.411869
jadeite              0.089293
dtype: float64

In [21]:
# Endmems -> Species

In [22]:
# %%timeit
mol_species_inv, rnorm = opt.nnls(species_stoic.T, mol_endmem)
mol_species_inv = pd.Series(mol_species_inv, index=species_stoic.index)
print('rnorm = ', rnorm)
mol_species_inv

rnorm =  0.0


diopside               0.000000
clinoenstatite         0.077828
hedenbergite           0.000000
alumino-buffonite      0.000000
buffonite              0.287121
essenite               0.040090
jadeite                0.000000
fe-aluminobuffonite    0.000000
fe-buffonite           0.000000
ca-tschermaks          0.293364
mg-tschermaks          0.000000
fe-tschermaks          0.078415
acmite                 0.089293
ferrosilite            0.133888
dtype: float64

In [23]:
species_stoic.T.dot(mol_species_inv)-mol_endmem

diopside             1.665335e-16
clinoenstatite      -1.110223e-16
hedenbergite         5.551115e-17
alumino-buffonite   -1.110223e-16
buffonite            8.326673e-17
essenite            -1.110223e-16
jadeite             -9.714451e-17
dtype: float64

In [24]:
phs.convert_species_to_comp(mol_spec)-mol_endmem

diopside             0.0
clinoenstatite       0.0
hedenbergite         0.0
alumino-buffonite    0.0
buffonite            0.0
essenite             0.0
jadeite              0.0
dtype: float64

In [25]:
phs.convert_endmember_comp(mol_endmem, output='moles_species')-mol_species_inv

diopside               0.0
clinoenstatite         0.0
hedenbergite           0.0
alumino-buffonite      0.0
buffonite              0.0
essenite               0.0
jadeite                0.0
fe-aluminobuffonite    0.0
fe-buffonite           0.0
ca-tschermaks          0.0
mg-tschermaks          0.0
fe-tschermaks          0.0
acmite                 0.0
ferrosilite            0.0
dtype: float64